In [1]:
#importing useful libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.core import Dense, Flatten
from keras.applications import imagenet_utils
from keras.applications import vgg19
from keras.models import Model

Using TensorFlow backend.


In [2]:
#loading model
img_rows = 224
img_cols = 224 
model=vgg19.VGG19(weights='imagenet',
                 include_top=False,
                 input_shape = (img_rows, img_cols, 3)
                 )
#trainable=false
for layer in model.layers:
    layer.trainable=False
    print(layer.__class__.__name__,layer.trainable)
#model summary
print(model.summary())

InputLayer False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
Conv2D False
Conv2D False
MaxPooling2D False
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block

In [3]:
def addModel(bottemModel,num_classes,classifier):
    top_model=bottemModel.output
    top_model=Flatten(name='flatten')(top_model)
    top_model=Dense(1048,activation='relu')(top_model)
    top_model=Dense(318,activation='relu')(top_model)
    top_model=Dense(97,activation='relu')(top_model)
    top_model=Dense(num_classes,activation=classifier)(top_model)
    return top_model

In [4]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

In [5]:
def createModel(model,top_model):
    x=Model(inputs=model.input,outputs=top_model)
    return x

In [6]:
#data genrator
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_data_dir = './5-celebrity-faces-dataset/data/train/'
validation_data_dir = './5-celebrity-faces-dataset/data/val/'
train_batchsize = 12
val_batchsize = 8
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_rows,img_cols),
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                  batch_size=train_batchsize,
                                                 )

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        class_mode='categorical',
        batch_size=val_batchsize,
        shuffle=False)

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [7]:
num_classes=5
classifier='softmax'
temp=addModel(model,num_classes,classifier)
new_model=createModel(model,temp)
#now new model is created

In [11]:

#checkpoints and data training
checkpoint = ModelCheckpoint("vgg19_face_5_cele.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)
callbacks = [earlystop, checkpoint]

new_model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])
batchsize=30
dataset_len=93
history = new_model.fit_generator(
    train_generator,
    epochs = 3,
    callbacks = callbacks,
    steps_per_epoch=dataset_len,
    validation_steps=25,
    validation_data = validation_generator)

new_model.save('vgg19_face_5_cele.h5')

Epoch 1/3
58/93 [=================>............] - ETA: 8:56 - loss: 2.6021 - accuracy: 0.9092

KeyboardInterrupt: 

In [10]:
#function to get valditaion accuracy
history.history['val_accuracy'][-1]

0.20000000298023224